## Creating your first LangChain project

In [1]:
pip install -q langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [3]:
import getpass
import os
if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass.getpass('Provide your Google API Key: ')

Provide your Google API Key: ··········


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [12]:
chat = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [13]:
messages = [
    (
        "system",
        "You are a helpful AI that helps the user make travel plans. Respond only in a single line.",
    ),
    ("human", "I want to go skiing. Which city should I go to?"),
]
first_msg = chat.invoke(messages)
first_msg

AIMessage(content='Consider Denver, Salt Lake City, or Vancouver for easy access to ski resorts.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--4433236d-8fd7-42bf-88da-6c7b1ab192f4-0', usage_metadata={'input_tokens': 33, 'output_tokens': 17, 'total_tokens': 50, 'input_token_details': {'cache_read': 0}})

In [14]:
print(first_msg.content)

Consider Denver, Salt Lake City, or Vancouver for easy access to ski resorts.


In [15]:
print(first_msg.usage_metadata["total_tokens"])

50


In [16]:
messages = [

    ("human", "What else can I do in that city?"),
]
ai_msg = chat.invoke(messages)
print(ai_msg.content)

Okay, I need more information to answer that question effectively!  Tell me:

*   **What city are you asking about?**
*   **What do you enjoy doing?**
*   **What is your budget?**
*   **What is the time of year/season?**
*   **Who are you travelling with?**

Once I have this information, I can give you some great suggestions.


## Prompt Templates

In [17]:
from langchain import PromptTemplate

dictionary_template = PromptTemplate(
    input_variables =['word'],
    template = "Give me a one line definition of {word}. Then, give one example of how it is used."
)
p = dictionary_template.format(word="Sesquipedalian")
print(p)


Give me a one line definition of Sesquipedalian. Then, give one example of how it is used.


In [18]:
chat = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [19]:
print(chat.invoke(p).content)

Sesquipedalian means characterized by long words; long-winded.
Example: "The professor's lectures were often sesquipedalian, leaving students struggling to grasp his meaning."


#### Multiple input variables

In [20]:
from langchain import PromptTemplate

define_translate = PromptTemplate(
    input_variables =['word','language'],
    template = "Give me a one line definition of {word}. Then, give the translation of that word in {language}."
)
p = define_translate.format(word="Sesquipedalian", language="Hindi")
print(p)
print(chat.invoke(p).content)

Give me a one line definition of Sesquipedalian. Then, give the translation of that word in Hindi.
Sesquipedalian means characterized by long words; long-winded.

Hindi translation: **दीर्घशब्दप्रयोगशील** (Dīrghashabdaprayogashīl)


#### Taking input from user

In [21]:
user_input = input("Enter the word you want to learn about: ")  # Use input() for text input

dictionary_template = PromptTemplate(
    input_variables=['word'],
    template="Give me a one line definition of {word}. Then, give one example of how it is used."
)
p = dictionary_template.format(word=user_input)
print(chat.invoke(p).content)

Enter the word you want to learn about: amazing
Amazing means causing great surprise or wonder; astonishing.

Example: "The magician's amazing disappearing act left the audience speechless."


#### ChatPromptTemplate

In [22]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that gives a one-line definition of the word entered by user"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(user_input="Sesquipedalian")
messages

[SystemMessage(content='You are a helpful assistant that gives a one-line definition of the word entered by user', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Sesquipedalian', additional_kwargs={}, response_metadata={})]

In [23]:
chat = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [24]:
print(chat.invoke(messages).content)

Using or characterized by long words.


#### Few shot prompt templates

In [25]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "rev": "I love this product",
        "answer": "positive"
    }, {
        "rev": "It was an average experience",
        "answer": "neutral"
    },{
        "rev": "I wonder why it is so highly rated.",
        "answer": "negative"
    }
]


# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["rev", "answer"],
    template= """
User: {rev}
AI: {answer}
"""
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = "Identify the sentiment of the user review. Here are some examples: "
# and the suffix our user input and output indicator
suffix = """
User: {review}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["review"],
    example_separator="\n"
)

user_review = "Well structured. Five stars."

print(few_shot_prompt_template.format(review=user_review))

Identify the sentiment of the user review. Here are some examples: 

User: I love this product
AI: positive


User: It was an average experience
AI: neutral


User: I wonder why it is so highly rated.
AI: negative


User: Well structured. Five stars.
AI: 


In [26]:
print(chat.invoke(few_shot_prompt_template.format(review=user_review)).content)


positive


## Chains

#### Generic chain - LLMChain

In [27]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI

dictionary_template = PromptTemplate(
    input_variables =["word"],
    template = "Give me a one line definition of {word}. Then, give one example of how it is used."
)

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

chain = LLMChain(llm=llm, prompt=dictionary_template)
chain.invoke("Anachronism")


<ipython-input-27-4ee3cdd59a4f>:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=dictionary_template)


{'word': 'Anachronism',
 'text': 'An anachronism is something that is out of place in time, either chronologically or contextually.\n\nExample: A cell phone appearing in a movie set in the 1800s.'}

In [28]:
input_list = [
    {"word": "Sesquipedalian"},
    {"word": "Anachronism"},
    {"word": "Onomatopoeia"}
]

chain.apply(input_list)

[{'text': 'Sesquipedalian means characterized by long words; long-winded.\n\nExample: "The professor\'s speech was so sesquipedalian that the students struggled to understand his points."'},
 {'text': 'Anachronism: Something that appears in the wrong time period.\n\nExample: A cell phone appearing in a movie set in ancient Rome.'},
 {'text': 'Onomatopoeia is the formation of a word from a sound associated with what is named.\n\nExample: The bees buzzed around the hive.'}]

#### Utility chains

In [29]:
from langchain.chains import LLMMathChain

In [30]:

calculator = LLMMathChain.from_llm(llm, verbose=True)
calculator.invoke("Calculate (625 raised to power (0.225))-(log10(100))")



> Entering new LLMMathChain chain...
Calculate (625 raised to power (0.225))-(log10(100))```text
625**0.225 - log10(100)
```
...numexpr.evaluate("625**0.225 - log10(100)")...
Answer: 2.2566996126039234
> Finished chain.


{'question': 'Calculate (625 raised to power (0.225))-(log10(100))',
 'answer': 'Answer: 2.2566996126039234'}

In [31]:
print(calculator.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [32]:

calc_llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash', temperature=0)
messages = [
    ("human", "Calculate (625 raised to power (0.225))-(log10(100))"),
]
calculation = calc_llm.invoke(messages)
print(calculation.content)

Here's how to solve this problem step-by-step:

**1. Calculate 625 raised to the power of 0.225:**

625<sup>0.225</sup> can be simplified by recognizing that 625 = 5<sup>4</sup>.  Therefore:

625<sup>0.225</sup> = (5<sup>4</sup>)<sup>0.225</sup> = 5<sup>(4 * 0.225)</sup> = 5<sup>1</sup> = 5

**2. Calculate log<sub>10</sub>(100):**

log<sub>10</sub>(100) = 2  (because 10<sup>2</sup> = 100)

**3. Subtract the results:**

5 - 2 = 3

Therefore, the answer is $\boxed{3}$


##### List of all chains - https://python.langchain.com/v0.1/docs/modules/chains/

### Sequential Chain

In [33]:
marketing_text = PromptTemplate(
    input_variables =['name','description'],
    template = "Generate a one line facebook ad copy for a product called {name}. Below is a description of this product: {description}"
)

text_llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash', temperature=0)

text_chain = LLMChain(llm=text_llm, prompt=marketing_text, output_key="copy_text")


translate_text = PromptTemplate(input_variables=["copy_text"], template="""Translate this text to Spanish:

{copy_text} """)

from langchain_google_genai import ChatGoogleGenerativeAI

translate_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite-001")

translate_chain = LLMChain(llm=translate_llm, prompt=translate_text, output_key="translated_copy")

from langchain.chains import SequentialChain

seq_chain = SequentialChain(
    chains = [text_chain, translate_chain],
    input_variables = ["name", "description"],
    output_variables = ["copy_text","translated_copy"]
)

seq_chain({"name": "AeroGlow Nightlight","description":"AeroGlow Nightlight is a smart, voice-activated nightlight that projects calming, animated constellations onto your ceiling."})

<ipython-input-33-0a5c6b7c1621>:29: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  seq_chain({"name": "AeroGlow Nightlight","description":"AeroGlow Nightlight is a smart, voice-activated nightlight that projects calming, animated constellations onto your ceiling."})


{'name': 'AeroGlow Nightlight',
 'description': 'AeroGlow Nightlight is a smart, voice-activated nightlight that projects calming, animated constellations onto your ceiling.',
 'copy_text': 'Transform your bedroom into a starry sanctuary with AeroGlow: the voice-activated nightlight that projects calming constellations onto your ceiling.',
 'translated_copy': "Here's the translation of the text to Spanish:\n\n**Transforma tu dormitorio en un santuario estrellado con AeroGlow: la luz nocturna activada por voz que proyecta constelaciones relajantes en tu techo.**\n\nHere's a breakdown of the translation choices:\n\n*   **Transform your bedroom:** Transforma tu dormitorio (or: Transforma tu habitación)\n*   **into a starry sanctuary:** en un santuario estrellado\n*   **with AeroGlow:** con AeroGlow (keeping the product name in English is common)\n*   **the voice-activated nightlight:** la luz nocturna activada por voz\n*   **that projects calming constellations:** que proyecta constelacio

In [34]:
result = seq_chain({"name": "AeroGlow Nightlight","description":"AeroGlow Nightlight is a smart, voice-activated nightlight that projects calming, animated constellations onto your ceiling."})
# Access the 'copy_text' from the results
print(result['copy_text'])
# Access the 'translated_copy' from the results
print(result['translated_copy'])

Transform your bedroom into a starry sanctuary with AeroGlow: the voice-activated nightlight that projects calming constellations onto your ceiling.
Here are a few options for translating the text to Spanish, with slightly different nuances:

**Option 1 (Most direct and clear):**

> Transforma tu dormitorio en un santuario estelar con AeroGlow: la luz nocturna activada por voz que proyecta constelaciones relajantes en tu techo.

**Option 2 (Emphasizing the calming effect):**

> Convierte tu habitación en un refugio de estrellas con AeroGlow: la luz nocturna con activación por voz que proyecta relajantes constelaciones sobre tu techo.

**Option 3 (Slightly more poetic):**

> Convierte tu dormitorio en un santuario de estrellas con AeroGlow: la luz nocturna activada por voz que pinta constelaciones tranquilizadoras en tu techo.

**Key changes and explanations:**

*   **"Transform" / "Convierte":** Both translate "transform." "Convierte" is a bit more common in this context.
*   **"Bedroo

### LCEL - Pipe operator and Runnables

In [35]:
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

dictionary_template = PromptTemplate(
    input_variables =["word"],
    template = "Give me a one line definition of {word}. Response should not contain the word - {word} itself, only the meaning."
)


llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')


chain = dictionary_template | llm | StrOutputParser()

In [36]:
chain.invoke({"word": "Onomatopoeia"})

'Words that imitate the sounds they describe.'

#### RunnablePassthrough, RunnableLambda and RunnableParallel

In [37]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [38]:
chain = RunnablePassthrough()

In [39]:
chain.invoke("abcd")

'abcd'

In [40]:
def output_length(input: str):
    output = len(input)
    return output

chain = RunnableLambda(output_length)
chain.invoke("input to output")

15

In [41]:
chain = RunnableParallel(text = RunnablePassthrough(), length = RunnableLambda(output_length))
chain.invoke("i love my teacher Conchita")

{'text': 'i love my teacher Conchita', 'length': 26}

#### Example

In [42]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

explainer_template = PromptTemplate(
    input_variables =["topic"],
    template = "Explain this topic in a single paragraph as if you were explaining it to a 10 year old - {topic}."
)

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

llm_chain = explainer_template | llm | StrOutputParser()
llm_chain.invoke({"topic": "Gravity"})

"Imagine the Earth is a giant magnet, but instead of attracting metal, it attracts *everything*!  That's gravity!  It's an invisible force that pulls everything towards the center of the Earth, keeping us stuck to the ground and making things fall down.  The bigger something is, the stronger its gravity.  That's why the Earth pulls you down, but the Earth also pulls on the moon, keeping it circling around us!  Even you have a tiny bit of gravity – it's just too weak to notice."

In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI

googlellm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

google_chain = explainer_template | googlellm | StrOutputParser()
google_chain.invoke({"topic": "Gravity"})

"Imagine everything in the universe has a superpower that pulls things towards it, like an invisible rope! That's gravity! Bigger things, like the Earth, have a stronger pull, so they keep us stuck to the ground. Gravity is why you don't float away, why apples fall from trees, and why the moon goes around the Earth instead of flying off into space. It's basically the force that keeps everything together!"

In [44]:
analyzer_template = PromptTemplate(
    input_variables =["topic", "explanation1", "explanation2"],
    template = """
    Which of the two explanations given below are better for explaining {topic} to 10 year old students.
    ###
    Explanation 1 - {explanation1}
    ###
    ***
    Explanation 2 - {explanation2}
    ***
    """
)

combined_chain = (
    RunnableParallel({"explanation1": llm_chain, "explanation2": google_chain, "topic": RunnablePassthrough()})
    | analyzer_template | llm_chain | StrOutputParser()
)

combined_chain.invoke({"topic": "Gravity"})

'Both explanations use good analogies to explain gravity, but Explanation 1 is better because it\'s more precise.  It correctly explains that gravity is a force, not just an invisible string, and it clarifies that bigger things have stronger gravity because they have more mass.  While Explanation 2\'s "invisible string" is a helpful image, it doesn\'t fully explain *why* bigger things pull harder, making Explanation 1 the clearer and more accurate explanation for 10-year-olds.'

In [45]:
analyzer_chain = analyzer_template | googlellm | StrOutputParser()

updated_chain = (
    RunnableParallel({"explanation1": llm_chain, "explanation2": google_chain, "topic": RunnablePassthrough()})
    | RunnableParallel({"input": RunnablePassthrough(), "analysis": analyzer_chain})
)

updated_chain.invoke({"topic": "Gravity"})

{'input': {'explanation1': "Imagine the Earth is like a giant magnet, but instead of attracting metal, it attracts *everything*!  That's gravity!  It's an invisible force that pulls everything towards the center of the Earth, keeping you stuck to the ground and making things fall down.  The bigger something is, the stronger its gravity.  That's why the Earth pulls you down, but you don't pull the Earth up towards you – you're just much smaller!  Gravity also keeps the moon going around the Earth, and the Earth going around the sun. It's like an invisible string holding everything together in space!",
  'explanation2': 'Imagine everything has a tiny bit of stickiness pulling it towards everything else. That stickiness is gravity! Big things, like the Earth, have a LOT of stickiness, so they pull everything towards them really strongly. That\'s why you stay on the ground instead of floating away, and why when you drop something, it falls down instead of up! The bigger something is, the m

In [46]:
result = updated_chain.invoke({"topic": "Gravity"})
# Access the 'copy_text' from the results
print(result['analysis'])

Both explanations are good and use age-appropriate analogies. However, here's a breakdown of why one might be slightly better:

*   **Explanation 1 (Trampoline Analogy):** This explanation is slightly better because it provides a more visual and tangible analogy. The trampoline analogy helps kids understand that gravity isn't just a pull, but also a distortion of space (although you don't need to use those words with 10-year-olds). They can easily visualize objects sinking into the trampoline. This also subtly hints at the relationship between mass and gravity (heavier objects create a deeper "sink").

*   **Explanation 2 (Invisible Rope Analogy):** This is a good analogy as well, but "invisible rope" isn't quite as intuitive or visually compelling as the trampoline. While it conveys the idea of a pulling force, it doesn't offer the same understanding of how mass influences gravity.

**In summary:**

Explanation 1 is slightly better because the trampoline analogy is more visually engag

#### Dynamic Routing

In [47]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain import PromptTemplate

classifier_template = PromptTemplate(
    input_variables = ["question", "answer"],
    template = """You are given a question and the user's response to that question. Classify the response as either `Correct`, or `Incorrect`.
    Do not respond with more than one word.
    Question - {question}
    User's Answer - {answer}
    Classification:"""
)

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

output_parser = StrOutputParser()

classifier_chain = classifier_template | llm | output_parser

classifier_chain.invoke({"question": "what are penguins", "answer": "Penguins are birds"})

'Correct'

In [48]:
correct_template = PromptTemplate(
    input_variables = ["question"],
    template = """The user was asked the following question and user answered it correctly. Now ask a more difficult question on the same topic to the user.
    Question: {question}
    New Question:"""
)

correct_chain = correct_template | llm | output_parser

incorrect_template = PromptTemplate(
    input_variables = ["question"],
    template = """The user was asked the following question and user answered it incorrectly. Give the correct answer and explain it to the user.
    Question: {question}
    Correct Answer:
    Explanation: """
)

incorrect_chain = incorrect_template | llm | output_parser

In [49]:
def route(info):
    if  info["result"].lower() == "correct":
        return correct_chain
    elif info["result"].lower() == "incorrect":
        return incorrect_chain
    else:
        return "Format is not correct"

In [50]:
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough


final_chain = RunnableParallel({"result": classifier_chain, "question": lambda x: x["question"], "answer": lambda x: x["answer"]}) | RunnableParallel({"response":RunnableLambda(route), "input": RunnablePassthrough()})

In [51]:
final_chain.invoke({"question": "what are penguins", "answer": "Penguins are sea animals"})

{'response': "Okay, since you know what penguins are, let's delve a little deeper.\n\n**New Question:** Beyond their physical adaptations for cold environments and aquatic life, what are some *behavioral* adaptations penguins have developed to survive and thrive in their challenging habitats, and how do these behaviors contribute to their overall success?",
 'input': {'result': 'Correct',
  'question': 'what are penguins',
  'answer': 'Penguins are sea animals'}}

In [52]:
final_chain.invoke({"question": "what are SQL joins", "answer": "SQL joins are operations used to combine rows from two or more tables based on a related column"})

{'response': "Okay, great! You understand the basics of SQL joins.\n\nLet's delve a bit deeper:\n\n**Question:**\n\nDescribe the key differences in terms of performance and use cases between using a `LEFT JOIN` with a `WHERE` clause that filters on the *right* table's columns *versus* using an `INNER JOIN` with the same filtering criteria applied to the *left* table's columns.  Explain why one approach might be significantly more efficient than the other in specific scenarios. Provide a brief example to illustrate your point.",
 'input': {'result': 'Correct',
  'question': 'what are SQL joins',
  'answer': 'SQL joins are operations used to combine rows from two or more tables based on a related column'}}

### Output Parsing

#### StrOutputParser()

In [53]:
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

dictionary_template = PromptTemplate(
    input_variables =["word"],
    template = "Give me a one line definition of {word}. Response should not contain the word itself, only the meaning."
)

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

chain = dictionary_template | llm
chain.invoke("Onomatopoeia")

AIMessage(content='Words that imitate or suggest the sounds they describe.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--864ee588-58e4-4f56-8a83-cb9d553c8f6a-0', usage_metadata={'input_tokens': 24, 'output_tokens': 11, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}})

In [54]:
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.output_parsers import StrOutputParser

dictionary_template = PromptTemplate(
    input_variables =["word"],
    template = "Give me a one line definition of {word}. Response should not contain the word itself, only the meaning."
)

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

chain = dictionary_template | llm | StrOutputParser()
chain.invoke("Onomatopoeia")

'Words that imitate the sounds they represent.'

#### StructuredOutputParser - Output as a specified Schema

In [55]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

response_schemas = [
    ResponseSchema(name="word", description="word entered by user"),
    ResponseSchema(name="meaning", description="One line meaning of the word given by user"),
    ResponseSchema(name="example", description="An example of how that word can be used in a line"),
    ResponseSchema(name="Etymology", description="Origin or history of the word")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"word": string  // word entered by user
	"meaning": string  // One line meaning of the word given by user
	"example": string  // An example of how that word can be used in a line
	"Etymology": string  // Origin or history of the word
}
```


In [56]:
prompt = PromptTemplate(
    template="Provide the meaning, an example of how the word is used in a sentence and the etymology of this word: {word}.\n{format_instructions}",
    input_variables=["word"],
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | chat_model | output_parser

chain.invoke("Onomatopoeia")

{'word': 'Onomatopoeia',
 'meaning': 'The formation of a word from a sound associated with what is named.',
 'example': 'The buzzing of the bee is a classic example of onomatopoeia.',
 'Etymology': "From Greek onomatopoiia, from onoma 'name' + poiein 'to make'."}

#### Comma separated list

In [57]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {topic}.\n{format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

chain = prompt | model | output_parser

In [58]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [59]:
chain.invoke({"topic": "healthy foods"})

['Apples', 'broccoli', 'salmon', 'quinoa', 'blueberries']

#### Date Time Parser

In [60]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [61]:
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1169-12-22T16:49:36.485008Z, 1059-10-06T19:39:03.639393Z, 1701-12-15T00:09:18.011909Z\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n{question}\n\n{format_instructions}')

In [62]:
chain = prompt | chat_model | output_parser

In [63]:
output = chain.invoke({"question": "When did India get independence?"})
print(output)

1947-08-15 00:00:00


## Memory

In [64]:
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

prompt_template = PromptTemplate(
    input_variables =["history", "new_input"],
    template = """You are having a chat with a human.
    previous chat: {history}
    Human: {new_input}
    Assistant: """
)

memory = ConversationBufferMemory()


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

chain = LLMChain(llm=llm, prompt=prompt_template, memory = memory, verbose=True)

chain.invoke({"new_input": "Which country is the biggest exporter of cotton?"})

<ipython-input-64-328bc8ba7278>:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()




> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with a human.
    previous chat: 
    Human: Which country is the biggest exporter of cotton?
    Assistant: 

> Finished chain.


{'new_input': 'Which country is the biggest exporter of cotton?',
 'history': '',
 'text': 'The United States.'}

In [65]:
chain.invoke({"new_input": "What is the total value of cotton exports of this country"})



> Entering new LLMChain chain...
Prompt after formatting:
You are having a chat with a human.
    previous chat: Human: Which country is the biggest exporter of cotton?
AI: The United States.
    Human: What is the total value of cotton exports of this country
    Assistant: 

> Finished chain.


{'new_input': 'What is the total value of cotton exports of this country',
 'history': 'Human: Which country is the biggest exporter of cotton?\nAI: The United States.',
 'text': 'That information changes from year to year. According to the USDA, in 2023/2024 the United States is expected to export $7.3 billion worth of cotton.'}

# RAG

### Document Loading

#### Text

In [66]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("LangchainRetrieval.txt")
loader.load()

[Document(metadata={'source': 'LangchainRetrieval.txt'}, page_content="Retrieval\nMany LLM applications require user-specific data that is not part of the model's training set. The primary way of accomplishing this is through Retrieval Augmented Generation (RAG). In this process, external data is retrieved and then passed to the LLM when doing the generation step.\n\nLangChain provides all the building blocks for RAG applications - from simple to complex. This section of the documentation covers everything related to the retrieval step - e.g. the fetching of the data. Although this sounds simple, it can be subtly complex. This encompasses several key modules.\n\nIllustrative diagram showing the data connection process with steps: Source, Load, Transform, Embed, Store, and Retrieve.\n\nDocument loaders\nDocument loaders load documents from many different sources. LangChain provides over 100 different document loaders as well as integrations with other major providers in the space, like 

In [67]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Excel+Course+Document.pdf")
pages = loader.load_and_split()

In [ ]:
pages[1]

Document(metadata={'producer': 'Skia/PDF m119 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Excel Course Document', 'source': 'Excel+Course+Document.pdf', 'total_pages': 8, 'page': 1, 'page_label': '2'}, page_content="Whatyou'll learn\n● ABeginner'sGuidetoMicrosoftExcel-MicrosoftExcel,LearnExcel,Spreadsheets,Formulas,Shortcuts,Macros● KnowledgeofalltheessentialExcelformulas● BecomeproﬁcientinExceldatatoolslikeSorting,Filtering,DatavalidationsandDataimporting● MasterExcel'smostpopularlookupfunctionssuchasVlookup,Hlookup,IndexandMatch● HarnessfullpotentialofExcelbycreatingPivottableswithslicers● MakegreatpresentationsusingtheConditionalandTableformattingoptions● VisuallyenchantviewersusingBarcharts,ScatterPlots,Histogramsetc.● IncreaseyourefﬁciencybylearninghowtocreateanduseimportantExcelshortcuts● ExplorefunandexcitingusecasesofExcel\nRequirements\nYouwillneedaPCwithanyversionofExcelinstalledinit\nWhothiscourseisfor\nAnyonecurioustomasterExcelfrombeginnertoAdv

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('./RAG/', glob="**/*.txt")

In [68]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=37334f9b64f1cc8bd58078e002206c6a873e0a7e57ce0a2a747c8925d0125c78
  St

In [69]:
docs = loader.load()

In [70]:
len(docs)

1

In [72]:
docs[0]

Document(metadata={'source': 'LangchainRetrieval.txt'}, page_content="Retrieval\nMany LLM applications require user-specific data that is not part of the model's training set. The primary way of accomplishing this is through Retrieval Augmented Generation (RAG). In this process, external data is retrieved and then passed to the LLM when doing the generation step.\n\nLangChain provides all the building blocks for RAG applications - from simple to complex. This section of the documentation covers everything related to the retrieval step - e.g. the fetching of the data. Although this sounds simple, it can be subtly complex. This encompasses several key modules.\n\nIllustrative diagram showing the data connection process with steps: Source, Load, Transform, Embed, Store, and Retrieve.\n\nDocument loaders\nDocument loaders load documents from many different sources. LangChain provides over 100 different document loaders as well as integrations with other major providers in the space, like A

In [ ]:
loader = DirectoryLoader('./RAG/', glob="**/*.txt", show_progress=True)
docs = loader.load()

100%|██████████| 4/4 [00:00<00:00, 42.02it/s]


In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='Movie_collection_dataset.csv')
data = loader.load()

In [ ]:
print(data)

[Document(metadata={'source': 'Movie_collection_dataset.csv', 'row': 0}, page_content='Collection: 48000\nMarketin_expense: 20.1264\nBudget: 36524.125\nLead_ Actor_Rating: 7.825\nLead_Actress_rating: 8.095\nTrailer_views: 527367\nGenre: Thriller\nNum_multiplex: 494\n3D_available: YES'), Document(metadata={'source': 'Movie_collection_dataset.csv', 'row': 1}, page_content='Collection: 43200\nMarketin_expense: 20.5462\nBudget: 35668.655\nLead_ Actor_Rating: 7.505\nLead_Actress_rating: 7.65\nTrailer_views: 494055\nGenre: Drama\nNum_multiplex: 462\n3D_available: NO'), Document(metadata={'source': 'Movie_collection_dataset.csv', 'row': 2}, page_content='Collection: 69400\nMarketin_expense: 20.5458\nBudget: 39912.675\nLead_ Actor_Rating: 7.485\nLead_Actress_rating: 7.57\nTrailer_views: 547051\nGenre: Comedy\nNum_multiplex: 458\n3D_available: NO'), Document(metadata={'source': 'Movie_collection_dataset.csv', 'row': 3}, page_content='Collection: 66800\nMarketin_expense: 20.6474\nBudget: 38873.8

### Splitting the document - Chunking

#### Recursively split by character

In [73]:
%pip install -qU langchain-text-splitters

In [74]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("LangchainRetrieval.txt")
text = loader.load()

In [75]:
text

[Document(metadata={'source': 'LangchainRetrieval.txt'}, page_content="Retrieval\nMany LLM applications require user-specific data that is not part of the model's training set. The primary way of accomplishing this is through Retrieval Augmented Generation (RAG). In this process, external data is retrieved and then passed to the LLM when doing the generation step.\n\nLangChain provides all the building blocks for RAG applications - from simple to complex. This section of the documentation covers everything related to the retrieval step - e.g. the fetching of the data. Although this sounds simple, it can be subtly complex. This encompasses several key modules.\n\nIllustrative diagram showing the data connection process with steps: Source, Load, Transform, Embed, Store, and Retrieve.\n\nDocument loaders\nDocument loaders load documents from many different sources. LangChain provides over 100 different document loaders as well as integrations with other major providers in the space, like 

In [76]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [77]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
)

In [78]:
texts = text_splitter.split_documents(text)
print(texts[0])
print(texts[1])
print(texts[2])

page_content='Retrieval' metadata={'source': 'LangchainRetrieval.txt'}
page_content='Many LLM applications require user-specific data that is not part of the model's training set. The primary way of accomplishing this is through Retrieval Augmented Generation (RAG). In this process,' metadata={'source': 'LangchainRetrieval.txt'}
page_content='In this process, external data is retrieved and then passed to the LLM when doing the generation step.' metadata={'source': 'LangchainRetrieval.txt'}


### Embedding

In [79]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [80]:
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [81]:
embeddings = embeddings_model.embed_documents(
    [
        "Hi",
        "What's up!",
        "Learning LangChain",
        "You should learn it from Conchita"
    ]
)
len(embeddings), len(embeddings[0])

(4, 768)

In [82]:
embeddings[0]

[0.03942074254155159,
 0.00341339991427958,
 -0.06045940890908241,
 -0.028973408043384552,
 0.06759811192750931,
 -0.0005555768148042262,
 0.025805775076150894,
 -0.028432175517082214,
 -0.012789401225745678,
 0.051125966012477875,
 -0.011636484414339066,
 0.025080760940909386,
 -0.020078590139746666,
 0.013835718855261803,
 -0.02999160625040531,
 -0.010279856622219086,
 0.012468338944017887,
 0.013220742344856262,
 0.04119221493601799,
 -0.022260591387748718,
 0.03646913915872574,
 0.014467360451817513,
 -0.006965399254113436,
 0.004088358022272587,
 0.039823830127716064,
 -0.03032483160495758,
 0.016167156398296356,
 -0.031830839812755585,
 -0.04246188700199127,
 0.004185791593044996,
 -0.05010295286774635,
 0.026064319536089897,
 -0.036640413105487823,
 0.021568095311522484,
 0.029510049149394035,
 -0.06420356035232544,
 -0.007808097172528505,
 0.028815818950533867,
 -0.009917114861309528,
 0.027429968118667603,
 0.006766802165657282,
 -0.046439699828624725,
 -0.048993658274412155,


In [83]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

[0.021657681092619896,
 -0.03848287835717201,
 -0.04262446612119675,
 -0.006191625725477934,
 0.012112059630453587]

### Vector Storage

#### Chroma

In [84]:
pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 5.3 MB/s eta 0:0

In [85]:
from langchain_community.document_loaders import TextLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader("LangchainRetrieval.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=20)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, embeddings_model)

In [86]:
query = "What is text embedding and how does langchain help in doing it"
docs = db.similarity_search(query)
print(docs[1].page_content)

Parent Document Retriever: This allows you to create multiple embeddings per parent document, allowing you to look up smaller chunks but return larger context.
Self Query Retriever: User questions often contain a reference to something that isn't just semantic but rather expresses some logic that can best be represented as a metadata filter. Self-query allows you to parse out the semantic part of a query from other metadata filters present in the query.
Ensemble Retriever: Sometimes you may want to retrieve documents from multiple different sources, or using multiple different algorithms. The ensemble retriever allows you to easily do this.
And more!
Indexing
The LangChain Indexing API syncs your data from any source into a vector store, helping you:


## Retrievers

In [87]:
from langchain_community.document_loaders import TextLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader("LangchainRetrieval.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, embeddings_model)

In [88]:
retriever = db.as_retriever()

In [89]:
docs = retriever.invoke("What is text embedding and how does langchain help in doing it")

In [90]:
len(docs)

4

In [91]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("What is text embedding and how does langchain help in doing it")

'Text embeddings capture the semantic meaning of text, allowing you to quickly and efficiently find other pieces of text that are similar. LangChain provides integrations with over 25 different embedding providers and methods, and provides a standard interface, allowing you to easily swap between models.'

In [92]:
retriever = db.as_retriever(search_kwargs={"k": 1})

In [93]:
docs = retriever.invoke("What is text embedding and how does langchain help in doing it")

In [94]:
docs

[Document(id='d0527d00-f37b-4f7e-84f9-becc4561323d', metadata={'source': 'LangchainRetrieval.txt'}, page_content='Text embedding models\nAnother key part of retrieval is creating embeddings for documents. Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of a text that are similar. LangChain provides integrations with over 25 different embedding providers and methods, from open-source to proprietary API, allowing you to choose the one best suited for your needs. LangChain provides a standard interface, allowing you to easily swap between models.')]

In [95]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.2}
)

In [96]:
docs = retriever.invoke("What is text embedding and how does langchain help in doing it")

In [97]:
docs

[Document(id='d0527d00-f37b-4f7e-84f9-becc4561323d', metadata={'source': 'LangchainRetrieval.txt'}, page_content='Text embedding models\nAnother key part of retrieval is creating embeddings for documents. Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of a text that are similar. LangChain provides integrations with over 25 different embedding providers and methods, from open-source to proprietary API, allowing you to choose the one best suited for your needs. LangChain provides a standard interface, allowing you to easily swap between models.'),
 Document(id='f5029671-c895-4aab-b1e0-266cc131d630', metadata={'source': 'LangchainRetrieval.txt'}, page_content='Another key part of retrieval is creating embeddings for documents. Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of a text that are similar. LangChain provides integrations with over 25 different embeddin

### Tools

#### Creating own custom tool

In [98]:
from langchain.tools import BaseTool, StructuredTool, tool

In [99]:
@tool
def name_of_tool(input: str) -> str:
    """Tool_Description"""
    return "Result"

In [100]:
print(name_of_tool.name)
print(name_of_tool.description)
print(name_of_tool.args)

name_of_tool
Tool_Description
{'input': {'title': 'Input', 'type': 'string'}}


#### Defining, Binding and Calling the tool

In [101]:
@tool
def conchita_coins(input: float) -> float:
    """Use this tool to convert USD to Conchita Academy coins"""
    return 1.3*(float(input))

In [102]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm_bind_tools = llm.bind_tools([conchita_coins])

In [103]:
result = llm_bind_tools.invoke("How many Conchita academy coins can I get for USD10")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'conchita_coins', 'arguments': '{"input": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--c507c6b7-7281-46ea-a7d2-000281224814-0', tool_calls=[{'name': 'conchita_coins', 'args': {'input': 10.0}, 'id': 'a93dc033-0921-42fa-a3be-5dab0c962bfe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 32, 'output_tokens': 6, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})

In [106]:
result.tool_calls

[{'name': 'conchita_coins',
  'args': {'input': 10.0},
  'id': 'a93dc033-0921-42fa-a3be-5dab0c962bfe',
  'type': 'tool_call'}]

In [107]:
tool_mapping = {
    'conchita_coins': conchita_coins
}
tool_mapping

{'conchita_coins': StructuredTool(name='conchita_coins', description='Use this tool to convert USD to Conchita Academy coins', args_schema=<class 'langchain_core.utils.pydantic.conchita_coins'>, func=<function conchita_coins at 0x7b31090a3c40>)}

In [108]:
tool = tool_mapping[result.tool_calls[0]["name"]]

In [109]:
tool_output = tool.invoke(result.tool_calls[0]["args"])

In [110]:
tool_output

13.0

### Using in-built tools
https://python.langchain.com/v0.1/docs/integrations/tools/

#### DuckDuckGo Search tool

In [112]:
%pip install --upgrade --quiet  duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.1 MB/s eta 0:00:00


In [113]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [114]:
search.name, search.description

('duckduckgo_search',
 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.')

In [115]:
search.run("What are tools in Langchain?")

'Types of LangChain Agents. Reactive Agents — Select and execute tools based on user input without long-term memory.; Conversational Agents — Maintain memory of past interactions, improving ... LangChain is a powerful tool that can be used to build a wide range of LLM-powered applications. It is simple to use and has a large user and contributor community. Conversational Agents : Build chatbots and virtual assistants that can engage in meaningful, context-aware conversations with users. In LangChain, an "Agent" is an AI entity that interacts with various "Tools" to perform tasks or answer queries. Tools are essentially functions that extend the agent\'s capabilities by ... How to build Custom Tools in LangChain 1: Using @tool decorator: There are several ways to build custom tools. The tool decorator is an easy way to create tools. In conclusion, LangChain\'s tools and agents represent a significant leap forward in the development of AI applications. By combining robust building blocks

In [116]:
from langchain_community.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
search.invoke("What are tools in Langchain?")

'snippet: What Are LangChain Agents? Agents in LangChain are advanced components that enable AI models to decide when and how to use tools dynamically., title: Understanding LangChain Tools and Agents: A Guide to Building ... - Medium, link: https://medium.com/@Shamimw/understanding-langchain-tools-and-agents-a-guide-to-building-smart-ai-applications-e81d200b3c12, snippet: LangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). It provides a standard interface for chains, many integrations with other tools, and end-to-end chains for common applications., title: Introduction to LangChain - GeeksforGeeks, link: https://www.geeksforgeeks.org/introduction-to-langchain/, snippet: Tools can utilize APIs and libraries for performing certain tasks.They can access data and gather information from sources. There are many built-in tools in LangChain for common tasks like doing ..., title: Mastering Tools and Tool Calling Agents i

### Wikipedia

In [117]:
%pip install --upgrade --quiet  wikipedia

  Preparing metadata (setup.py) ... done


In [118]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikipedia.invoke("LangChain")

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Intelligent agent\nSummary: In artificial intelligence, an intelligent agent is an entity that perceives its environment, takes actions autonomously to achieve goals, and may improve its performance through machine learning or by acquiring knowledge. Leading AI textbooks define artificial intelligence as the "study and design of intelligent agents," emphasizing that goal-directed behavior is central to intelligence.\nA specialized subset of intelligent agents, agentic AI (also known as an AI agent or simply agent), expands this concept by proactively pursuing goals, making decisions, and taking actions over extended periods, ther

### Agents

In [129]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(

      [  (
            "system",
            "You are a helpful assistant. For answering the user query, look for information using DuckDuckGo Search and Wikipedia and then give the final answer",
        ),
        ("human", "{input}"),


           ("placeholder", "{agent_scratchpad}"),

      ]
)
tools = [search, wikipedia]

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [130]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [131]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [132]:
agent_executor.invoke({"input": "When was Nelson Mandela born and what are some of his famous quotes?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Nelson Mandela'}`


Page: Nelson Mandela
Summary: Nelson Rolihlahla Mandela ( man-DEL-ə, Xhosa: [xolíɬaɬa mandɛ̂ːla]; born Rolihlahla Mandela; 18 July 1918 – 5 December 2013) was a South African anti-apartheid activist and politician who served as the first president of South Africa from 1994 to 1999. He was the country's first black head of state and the first elected in a fully representative democratic election. His government focused on dismantling the legacy of apartheid by fostering racial reconciliation. Ideologically an African nationalist and socialist, he served as the president of the African National Congress (ANC) party from 1991 to 1997.
A Xhosa, Mandela was born into the Thembu royal family in Mvezo, South Africa. He studied law at the University of Fort Hare and the University of Witwatersrand before working as a lawyer in Johannesburg. There he became involved in anti-colonial and African n

{'input': 'When was Nelson Mandela born and what are some of his famous quotes?',
 'output': 'Nelson Mandela was born on July 18, 1918.\n\nHere are some of his famous quotes:\n\n*   "Education is the most powerful weapon which you can use to change the world."\n*   "For to be free is not merely to cast off one\'s chains, but to live in a way that respects and enhances the freedom of others."\n*   "The greatest glory in living lies not in never falling, but in rising every time we fall."\n*   "I learned that courage was not the absence of fear, but the triumph over it. The brave man is not he who does not feel afraid, but he who conquers that fear."\n*   "No one is born hating another person because of the color of his skin, or his background, or his religion. People must learn to hate, and if they can learn to hate, they can be taught to love, for love comes more naturally to the human heart than its opposite."'}

#### Agent with memory

In [133]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Based on user query and the chat history, look for information using DuckDuckGo Search and Wikipedia and then give the final answer",
        ),
        ("placeholder", "{history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [134]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [135]:
agent_with_history.invoke(
    {"input": "When was Nelson Mandela born?"},
    config={"configurable": {"session_id": "sess1"}},
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Nelson Mandela'}`


Page: Nelson Mandela
Summary: Nelson Rolihlahla Mandela ( man-DEL-ə, Xhosa: [xolíɬaɬa mandɛ̂ːla]; born Rolihlahla Mandela; 18 July 1918 – 5 December 2013) was a South African anti-apartheid activist and politician who served as the first president of South Africa from 1994 to 1999. He was the country's first black head of state and the first elected in a fully representative democratic election. His government focused on dismantling the legacy of apartheid by fostering racial reconciliation. Ideologically an African nationalist and socialist, he served as the president of the African National Congress (ANC) party from 1991 to 1997.
A Xhosa, Mandela was born into the Thembu royal family in Mvezo, South Africa. He studied law at the University of Fort Hare and the University of Witwatersrand before working as a lawyer in Johannesburg. There he became involved in anti-colonial and African n

{'input': 'When was Nelson Mandela born?',
 'history': [],
 'output': 'Nelson Mandela was born on 18 July 1918.\n'}

In [136]:
result = agent_with_history.invoke(
    {"input": "What are some of his famous quotes?"},
    config={"configurable": {"session_id": "sess1"}},
)



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'famous quotes by Nelson Mandela'}`


snippet: Nelson Mandela's Powerful Quotes. Here are 95 inspiring quotes by Nelson Mandela, touching on topics like courage, leadership, forgiveness, and perseverance: On Courage and Strength • "Courage is not the absence of fear — it's inspiring others to move beyond it.", title: 95 Nelson Mandela Powerful Quotes to Inspire and Guide You, link: https://lifeispositive.com/mandela-quotes/, snippet: Nelson Mandela was a brave leader and a symbol of hope for many people around the world. He fought against unfair treatment and worked hard for the freedom of his country, South Africa. Mandela's words continue to inspire millions, reminding us to believe in ourselves and to fight for justice. His quotes often reflect themes, title: Top 150 Nelson Mandela Quotes (With Meanings) - Scripture Savvy, link: https://scripturesavvy.com/nelson-mandela-quotes/, snippet: Thi

In [137]:
result['output']

'Here are some of Nelson Mandela\'s most famous quotes:\n\n*   "Courage is not the absence of fear — it\'s inspiring others to move beyond it."\n*   "Education is the most powerful weapon which you can use to change the world."\n*   "It always seems impossible until it\'s done."'